<a href="https://colab.research.google.com/github/slavyolov/Algorithms/blob/main/Statistics_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [303]:
import warnings

# Code block where warnings are ignored
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")  # Ignore all warnings

In [73]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
!ls

drive  sample_data


# Assignment details
## **Task:**
Use the provided datasets to perform the homogeneity test for one or more segments using the risk variable granted amount.

## **Data:**
Snapshots of all clients/facilities at the beginning and end of the observation date (observation date, identifiers, exposure amounts, default information, segmentation information, model information, …). For a data dictionary help table please see Appendix section.

## **Background:**
The purpose of the test to ensure that the proposed model does not group exposures of significantly different risk characteristics (w.r.t the predicted parameter) in the same rating grade or pool. The test compares for each combination of rating grade/pool and bin of the considered risk variable the observed default rate of exposures within the bin to outside the bin. If a risk driver is not categorical and not already binned, then it is binned into tertiles (low, medium, high). **A significance level of 2.5% is set for this test.**


# **Homogenity test**
A homogeneity test is used to determine whether two or more groups or populations are homogeneous or similar with respect to a particular characteristic or variable. The test helps to assess whether the observed differences between groups are statistically significant or if they can be attributed to random chance.

**The chi-squared homogeneity test answers the question do these two (or more) samples come from the same population?**

# **Setting the hypothesis (regarding the homogeneity of the groups or populations)**
- null hypothesis (H₀) -  assumes that there is no significant difference between the groups
- alternative hypothesis (H₁) - suggests that there is a significant difference.

# **Data analysis**

## Read and display the data

In [100]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Statistics/PD_Data_prep.csv")
df.drop(columns=["Unnamed: 0"], inplace=True)
df.head()
# print(df.info())
# print(df.describe().T)

,cohort_obs_dt,segment_calibration,consolidated_id,default_first_in_12M_dt,pd_model_class,pd_model_pct,pd_decided_class,pd_decided_pct,issued_or_outstanding_amt_EUR,granted_amt_EUR,EaD_amt_EUR,RWA_amt_EUR,Validation_sample
0,12/31/2021,MC,894649,NaN,3,0.002172,3,0.002172,10719602.04,11253399.26,2248320.40,830873.37,1
1,12/31/2022,MC,156028,NaN,6,0.024611,6,0.024611,10909561.43,11578473.65,2286312.29,1988221.54,1
2,12/31/2021,LC,89270,NaN,5,0.010027,5,0.010027,610341.92,1610341.92,361641.92,194084.49,1
3,12/31/2022,LC,995075,NaN,2,0.001260,2,0.001260,634525.00,1635000.00,238205.00,50170.38,1
4,12/31/2021,LC,629963,NaN,4,0.004863,4,0.004863,156861.64,300000.00,31372.33,30385.33,1


In [101]:
# We have two segments -> Therefore we must implement two homogenity test (1 for LC and 1 for MC)
df.groupby('segment_calibration')['granted_amt_EUR'].count()

segment_calibration
LC    5071
MC    7008
Name: granted_amt_EUR, dtype: int64

In [102]:
# We have two segments -> Therefore we must implement two homogenity test (1 for LC and 1 for MC)
df.groupby(['pd_model_class', 'segment_calibration'])['granted_amt_EUR'].count()

pd_model_class  segment_calibration
1               LC                      166
                MC                      199
2               LC                      726
                MC                      863
3               LC                     1033
                MC                     1613
4               LC                     1135
                MC                     1470
5               LC                      880
                MC                     1289
6               LC                      540
                MC                      724
7               LC                      325
                MC                      411
8               LC                      106
                MC                      125
9               LC                       60
                MC                      153
10              LC                      100
                MC                      161
Name: granted_amt_EUR, dtype: int64

## Add tertile column

In [103]:
# Add tertiles column based on

# Step 1 : Group the observations from each rating grade by some relevant variable (granted amount)

#TODO: understand which one - maybe ask them for support
# rating grades : 'pd_model_class' OR 'pd_decided_class'
# relevant variable : (outstanding/granted amount) -> 	'issued_or_outstanding_amt_EUR'	OR 'granted_amt_EUR'

# significance level (alpha)
significance_level = 0.025

# Create tertiles for each group
df['tertile'] = df.groupby(['pd_model_class', 'segment_calibration'])['granted_amt_EUR']\
    .transform(lambda x: pd.qcut(x, q=3, labels=["Low", "Medium", "High"]))

df.head()

,cohort_obs_dt,segment_calibration,consolidated_id,default_first_in_12M_dt,pd_model_class,pd_model_pct,pd_decided_class,pd_decided_pct,issued_or_outstanding_amt_EUR,granted_amt_EUR,EaD_amt_EUR,RWA_amt_EUR,Validation_sample,tertile
0,12/31/2021,MC,894649,NaN,3,0.002172,3,0.002172,10719602.04,11253399.26,2248320.40,830873.37,1,High
1,12/31/2022,MC,156028,NaN,6,0.024611,6,0.024611,10909561.43,11578473.65,2286312.29,1988221.54,1,High
2,12/31/2021,LC,89270,NaN,5,0.010027,5,0.010027,610341.92,1610341.92,361641.92,194084.49,1,Medium
3,12/31/2022,LC,995075,NaN,2,0.001260,2,0.001260,634525.00,1635000.00,238205.00,50170.38,1,Medium
4,12/31/2021,LC,629963,NaN,4,0.004863,4,0.004863,156861.64,300000.00,31372.33,30385.33,1,Low


In [104]:
# tertiles_manual_check (check the excel - for a nice table overview)
df.to_csv("/content/drive/MyDrive/Statistics/tertiles_manual_check.csv")

In [154]:
pd.read_csv("/content/drive/MyDrive/Statistics/demo_data_to_test_dataprep.csv")

,pd_model_class,default_first_in_12M_dt,tertile,segment_calibration
0,1,1.0,High,CL
1,1,1.0,High,CL
2,1,1.0,High,CL
3,1,1.0,High,CL
4,1,1.0,High,CL
...,...,...,...,...
3227,2,NaN,Low,ML
3228,2,1.0,Medium,ML
3229,2,NaN,Medium,ML
3230,2,1.0,High,ML


In [157]:
# test with demo data to verify aggregations
df = pd.read_csv("/content/drive/MyDrive/Statistics/demo_data_to_test_dataprep.csv")
df.head()


,pd_model_class,default_first_in_12M_dt,tertile,segment_calibration
0,1,1.0,High,CL
1,1,1.0,High,CL
2,1,1.0,High,CL
3,1,1.0,High,CL
4,1,1.0,High,CL


## Number of defaults (in bin) data

In [158]:
# Calculate the number of defaults (default rate)
agg_df = df.groupby(['pd_model_class', 'segment_calibration', 'tertile']).agg(
    D_in_bin=('default_first_in_12M_dt', lambda x: x.notna().sum()),
    Not_D_in_bin=('default_first_in_12M_dt', lambda x: x.isna().sum()),
    Observations_in_bin=('pd_model_class', lambda x: x.count())
)

agg_df["DR"] = agg_df["D_in_bin"] / agg_df["Observations_in_bin"]
agg_df = agg_df.reset_index()
agg_df.head(6) # display

,pd_model_class,segment_calibration,tertile,D_in_bin,Not_D_in_bin,Observations_in_bin,DR
0,1,CL,High,181,596,777,0.232947
1,1,CL,Low,299,1164,1463,0.204375
2,1,CL,Medium,207,779,986,0.209939
3,2,ML,High,1,1,2,0.500000
4,2,ML,Low,1,1,2,0.500000
5,2,ML,Medium,1,1,2,0.500000


## Add the Out of bin data

In [159]:
total = df.groupby(['pd_model_class', 'segment_calibration']).agg(
    defaulted=('default_first_in_12M_dt', lambda x: x.notna().sum()),
    not_defaulted=('default_first_in_12M_dt', lambda x: x.isna().sum()),
    Observations_in_bin=('pd_model_class', lambda x: x.count())
)
total.columns = ["Tot_defaulted", "Tot_not_defaulted", "Tot_observations"]
total = total.reset_index()
total.head(4)

,pd_model_class,segment_calibration,Tot_defaulted,Tot_not_defaulted,Tot_observations
0,1,CL,687,2539,3226
1,2,ML,3,3,6


In [160]:
agg_df_n = agg_df.merge(total, on=["pd_model_class", "segment_calibration"])
agg_df_n["D_out_of_bin"] = agg_df_n["Tot_defaulted"] - agg_df_n["D_in_bin"]
agg_df_n["Not_D_out_of_bin"] = agg_df_n["Tot_not_defaulted"] - agg_df_n["Not_D_in_bin"]
agg_df_n["Tot_out_of_bin"] = agg_df_n["Tot_observations"] - agg_df_n["Observations_in_bin"]
agg_df_n["DR_out_of_bin"] = agg_df_n["D_out_of_bin"] / agg_df_n["Tot_out_of_bin"]
# agg_df_n = agg_df_n[["pd_model_class", "segment_calibration", "segment_calibration", "tertile", "not_defaulted", "defaulted",
#                      "default_rate", "N_out_of_bin", "D_out_of_bin", "DR_out_of_bin"]]
agg_df_n.head(6)

# # Perform the pivot operation
# pivoted_df = agg_df_n.pivot(index=['tertile', 'not_defaulted', 'defaulted', 'default_rate',
#                                    'N_out_of_bin', 'D_out_of_bin', 'DR_out_of_bin'],
#                       columns='pd_model_class',
#                       values=['tertile'])

# # Reset the index to convert the rows into columns
# pivoted_df = pivoted_df.reset_index()
# pivoted_df

,pd_model_class,segment_calibration,tertile,D_in_bin,Not_D_in_bin,Observations_in_bin,DR,Tot_defaulted,Tot_not_defaulted,Tot_observations,D_out_of_bin,Not_D_out_of_bin,Tot_out_of_bin,DR_out_of_bin
0,1,CL,High,181,596,777,0.232947,687,2539,3226,506,1943,2449,0.206615
1,1,CL,Low,299,1164,1463,0.204375,687,2539,3226,388,1375,1763,0.220079
2,1,CL,Medium,207,779,986,0.209939,687,2539,3226,480,1760,2240,0.214286
3,2,ML,High,1,1,2,0.500000,3,3,6,2,2,4,0.500000
4,2,ML,Low,1,1,2,0.500000,3,3,6,2,2,4,0.500000
5,2,ML,Medium,1,1,2,0.500000,3,3,6,2,2,4,0.500000


In [260]:
agg_df_n.to_csv("/content/drive/MyDrive/Statistics/output2.csv")

In [261]:
agg_df_n

,pd_model_class,segment_calibration,tertile,D_in_bin,Not_D_in_bin,Observations_in_bin,DR,Tot_defaulted,Tot_not_defaulted,Tot_observations,D_out_of_bin,Not_D_out_of_bin,Tot_out_of_bin,DR_out_of_bin
0,1,CL,High,181,596,777,0.232947,687,2539,3226,506,1943,2449,0.206615
1,1,CL,Low,299,1164,1463,0.204375,687,2539,3226,388,1375,1763,0.220079
2,1,CL,Medium,207,779,986,0.209939,687,2539,3226,480,1760,2240,0.214286
3,2,ML,High,1,1,2,0.500000,3,3,6,2,2,4,0.500000
4,2,ML,Low,1,1,2,0.500000,3,3,6,2,2,4,0.500000
5,2,ML,Medium,1,1,2,0.500000,3,3,6,2,2,4,0.500000


# Run homogenity test

## **Function to calculate the homogenity test**

In [287]:
import numpy as np
from scipy.stats import chisquare
from scipy.stats import chi2

def calculate_homogenity_test(observations):
    """
    This functions covers the homogenity test implementation described here :
        https://courses.lumenlearning.com/wm-concepts-statistics/chapter/test-of-homogeneity/
    The functions is inspired from :
        https://rowannicholls.github.io/python/statistics/hypothesis_testing/chi_squared.html#homogeneity-test

    Testing if the different samples come from the same population

    params: observations - it could be a matrix of any kind but tested
        with 2x2 matrix

    Returns:
        p_value that helps to accept/reject a hypothesis
    """
    row_totals = np.array([np.sum(observations, axis=1)])
    col_totals = np.array([np.sum(observations, axis=0)])

    n = np.sum(observations)

    # Calculate the expected observations
    expected = np.dot(row_totals.T, col_totals) / n

    # Calculate chi-square test statistic
    chisq, _ = chisquare(observations, expected)
    chisq = np.sum(chisq)

    # Degrees of freedom
    rows = observations.shape[0]
    cols = observations.shape[1]
    df = (rows - 1) * (cols - 1)

    # Convert chi-square test statistic to p-value
    p = 1 - chi2.cdf(chisq, df)

    # Uncomment to display the value
    # print(f'p = {p:5.3f}')
    return f'{p:5.3f}'
    # return p

## Demo homogenity test

In [283]:
# 1st row of each key in the dictionary shows the defaulted (in/out of bin)
# 2nd row of each key in the dictionary shows the non_defaulted (in/out of bin)
observations_dict ={"High": np.array(
    [
        [181, 506],
        [596, 1943]
    ]
),
        "Mid": np.array(
    [
        [207, 779],
        [480, 1760]
    ]
),
        "Low": np.array(
    [
        [299, 388],
        [1164, 1375]
    ]
)}

for observation_k, observation_v in observations_dict.items():
    print(observation_k, ":", calculate_homogenity_test(observation_v))
    print(25*"=")

High : 0.11825839261835613
Mid : 0.7811663883448277
Low : 0.27806054194835705


## Tun homogenity test on assesment data

In [284]:
segment_calibration = list(agg_df_n["segment_calibration"].unique())
pd_model_class = list(agg_df_n["pd_model_class"].unique())
level = list(agg_df_n["tertile"].unique())

combinations = list(itertools.product(segment_calibration, pd_model_class, level))

for combination in combinations:
    print(combination)

('CL', 1, 'High')
('CL', 1, 'Low')
('CL', 1, 'Medium')
('CL', 2, 'High')
('CL', 2, 'Low')
('CL', 2, 'Medium')
('ML', 1, 'High')
('ML', 1, 'Low')
('ML', 1, 'Medium')
('ML', 2, 'High')
('ML', 2, 'Low')
('ML', 2, 'Medium')


In [305]:
import numpy as np
import itertools

observations = {}

tmp_df = agg_df_n.copy(deep=True)
for combination in combinations:
    l = (
        tmp_df[["D_in_bin", "D_out_of_bin"]][
        (tmp_df["segment_calibration"]==combination[0]) &
        (tmp_df["pd_model_class"]==combination[1]) &
        (tmp_df["tertile"]==combination[2])
                    ].values.tolist()
    )

    flattened_list = list(itertools.chain(*l))

    l2 = (
        tmp_df[["Not_D_in_bin", "Not_D_out_of_bin"]][
        (tmp_df["segment_calibration"]==combination[0]) &
        (tmp_df["pd_model_class"]==combination[1]) &
        (tmp_df["tertile"]==combination[2])
                    ].values.tolist()
    )

    flattened_list_2 = list(itertools.chain(*l2))

    # print(combination)
    # print(flattened_list)
    # print(flattened_list_2)

    observation = np.array(
      [
          flattened_list,
          flattened_list_2
      ]
      )

    observations[combination] = observation

# Append the p_value to empty data frame
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
result_df = pd.DataFrame(columns=['segment_calibration', 'pd_model_class',
                                  'tertile', 'p_value'])

for observation_k, observation_v in observations.items():
    if observation_v.size:
        print(observation_k, ":", calculate_homogenity_test(observation_v))
        print(25*"=")
        result_df = result_df.append(
            {"segment_calibration" : observation_k[0],
             "pd_model_class" : observation_k[1],
             "tertile" : observation_k[2],
             "p_value" : calculate_homogenity_test(observation_v)
                            }, ignore_index=True)

('CL', 1, 'High') : 0.118
('CL', 1, 'Low') : 0.278
('CL', 1, 'Medium') : 0.781
('ML', 2, 'High') : 1.000
('ML', 2, 'Low') : 1.000
('ML', 2, 'Medium') : 1.000


In [306]:
result_df

,segment_calibration,pd_model_class,tertile,p_value
0,CL,1,High,0.118
1,CL,1,Low,0.278
2,CL,1,Medium,0.781
3,ML,2,High,1.000
4,ML,2,Low,1.000
5,ML,2,Medium,1.000


## Combine data frames and return final df

In [ ]:
# combine result_df and the previous table to return the needed result